# Imports and incantations

In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
pd.set_option('display.max_columns', None)  # Unlimited columns

In [2]:
sample_submission = pd.read_csv('kaggle_data/sample_submission.csv')
test_features = pd.read_csv('kaggle_data/test_features.csv')
train_features = pd.read_csv('kaggle_data/train_features.csv')
train_labels = pd.read_csv('kaggle_data/train_labels.csv')

In [3]:
print(train_features.shape)
train_features.head()

(37745, 103)


,id,member_id,loan_amnt,funded_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,application_type,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,disbursement_method
0,43373,NaN,12000,12000,36 months,16.02%,422.01,C,C5,Driver,4 years,MORTGAGE,81000.0,NaN,NaN,debt_consolidation,Debt consolidation,280xx,NC,12.76,0,Dec-2005,1,48.0,NaN,7,0,4912,23.5%,15,w,0,48.0,Individual,NaN,NaN,0,0,174783,1,2,0,4,13.0,33668,82.0,2,5,2962,63.0,20900,1,1,2,9,24969.0,588.0,89.3,0,0,145.0,23,1,1,1,8.0,NaN,1.0,NaN,2,2,2,2,2,9,4,5,2,7,0.0,0,0,2,92.9,100.0,0,0,206618,38580,5500,40863,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
1,24414,NaN,6300,6300,36 months,14.07%,215.54,C,C3,GMP Lead,4 years,RENT,39000.0,NaN,NaN,debt_consolidation,Debt consolidation,920xx,CA,21.42,0,Mar-2012,1,NaN,NaN,10,0,11876,59.4%,12,w,0,NaN,Individual,NaN,NaN,0,0,31046,0,1,0,2,14.0,19170,83.0,3,8,5153,72.0,20000,1,0,1,10,3105.0,6877.0,59.3,0,0,17.0,72,8,8,0,8.0,NaN,6.0,NaN,0,6,7,7,8,2,9,10,7,10,0.0,0,0,3,100.0,42.9,0,0,43140,31046,16900,23140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
2,46723,NaN,4500,4500,36 months,7.21%,139.38,A,A3,Accounts,3 years,RENT,78000.0,NaN,NaN,debt_consolidation,Debt consolidation,906xx,CA,2.17,0,May-2000,2,NaN,NaN,13,0,1715,5.2%,19,w,0,NaN,Individual,NaN,NaN,0,0,35329,0,8,1,2,10.0,33614,106.0,1,1,1196,54.0,33300,4,0,3,3,2718.0,21585.0,7.4,0,0,166.0,218,10,10,0,31.0,NaN,1.0,NaN,0,3,3,3,4,13,5,6,3,13,0.0,0,0,2,100.0,0.0,0,0,65092,35329,23300,31792,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
3,24878,NaN,12000,12000,36 months,9.44%,384.06,B,B1,Client Processing Associate,10+ years,MORTGAGE,62000.0,NaN,NaN,debt_consolidation,Debt consolidation,441xx,OH,13.76,0,Feb-2003,0,NaN,NaN,22,0,9404,16.3%,33,w,0,NaN,Individual,NaN,NaN,0,0,75160,4,1,0,2,13.0,17123,67.0,6,9,6898,32.0,57600,3,1,2,12,3579.0,33602.0,17.0,0,0,162.0,179,1,1,1,1.0,NaN,5.0,NaN,0,1,3,9,13,7,20,25,3,22,0.0,0,0,7,100.0,0.0,0,0,133065,26527,40500,25465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
4,39038,NaN,12000,12000,36 months,22.35%,460.47,D,D5,Construction Operator,10+ years,MORTGAGE,70000.0,NaN,NaN,debt_consolidation,Debt consolidation,751xx,TX,14.04,4,Nov-2001,2,13.0,NaN,18,0,17895,24.2%,45,w,0,13.0,Individual,NaN,NaN,0,7218,113702,9,2,2,4,6.0,16711,68.0,9,10,5373,35.0,73900,3,6,3,14,6317.0,32863.0,24.1,0,0,159.0,201,0,0,1,0.0,62.0,3.0,62.0,6,9,11,11,14,22,15,20,11,18,0.0,0,2,11,82.2,9.1,0,0,192842,34606,43300,24499,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash


# Model 1: Baseline
1. I use all the numerical features that are already null-free.
2. I use GridSearchCV to find the optimal depth of decision tree.

In [4]:
# First, create an array of features X1 that contains only numerical columns with no Nulls.
X1 = train_features.dropna(axis='columns', how='any')
X1 = X1.select_dtypes(include='number')
X1.shape

(37745, 53)

In [5]:
# Define the vector y_train of labels for the training data
y_train = train_labels['charged_off']

In [6]:
# Create a pipeline, such that this workflow can be easily expanded in later sections
pipe = make_pipeline(
    DecisionTreeClassifier()
)
parameters = {'decisiontreeclassifier__max_depth':[1,2,3,4]}
gs = GridSearchCV(estimator=pipe, 
                  param_grid=parameters, 
                  cv=3, 
                  n_jobs=-1,
                  return_train_score=False)

In [7]:
%%time
gs.fit(X1,y_train)

CPU times: user 262 ms, sys: 131 ms, total: 393 ms
Wall time: 4.63 s


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('decisiontreeclassifier', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'decisiontreeclassifier__max_depth': [1, 2, 3, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
       scoring=None, verbose=0)

In [8]:
print(f'Best parameters: {gs.best_params_}')
print(f'Best score: {gs.best_score_:0.4f}')

Best parameters: {'decisiontreeclassifier__max_depth': 1}
Best score: 0.8529


# Model 2: Cleaner data
I want to apply the same cleaning operations to train and test data, so I will write it into functions.  I'll be careful about how those functions depend on attributes that may be unique to the training or testing dataset, to make sure the two cleaned datasets have the same columns. This notebook contains the steps where I figure outh what cleanup operations to perform, and then a single function that actually carries them all out.

In [9]:
train_features.shape

(37745, 103)

In [10]:
# Several columns are nothing but NANs.
train_features.isnull().all()[train_features.isnull().all() == True]

member_id    True
url          True
desc         True
dtype: bool

In [11]:
# Several of the non-numeric features are actually numbers parsed as strings.
train_features.select_dtypes(exclude='number')

,term,int_rate,grade,sub_grade,emp_title,emp_length,home_ownership,purpose,title,zip_code,addr_state,earliest_cr_line,revol_util,initial_list_status,application_type,sec_app_earliest_cr_line,disbursement_method
0,36 months,16.02%,C,C5,Driver,4 years,MORTGAGE,debt_consolidation,Debt consolidation,280xx,NC,Dec-2005,23.5%,w,Individual,NaN,Cash
1,36 months,14.07%,C,C3,GMP Lead,4 years,RENT,debt_consolidation,Debt consolidation,920xx,CA,Mar-2012,59.4%,w,Individual,NaN,Cash
2,36 months,7.21%,A,A3,Accounts,3 years,RENT,debt_consolidation,Debt consolidation,906xx,CA,May-2000,5.2%,w,Individual,NaN,Cash
3,36 months,9.44%,B,B1,Client Processing Associate,10+ years,MORTGAGE,debt_consolidation,Debt consolidation,441xx,OH,Feb-2003,16.3%,w,Individual,NaN,Cash
4,36 months,22.35%,D,D5,Construction Operator,10+ years,MORTGAGE,debt_consolidation,Debt consolidation,751xx,TX,Nov-2001,24.2%,w,Individual,NaN,Cash
5,36 months,15.49%,C,C4,AML aNalyst,< 1 year,RENT,home_improvement,Home improvement,112xx,NY,Aug-2011,0%,w,Individual,NaN,Cash
6,60 months,12.73%,B,B5,NaN,< 1 year,MORTGAGE,other,Other,276xx,NC,Sep-2010,41.7%,w,Individual,NaN,Cash
7,36 months,11.99%,B,B5,Driver,2 years,MORTGAGE,other,Other,286xx,NC,Mar-2005,51.1%,f,Individual,NaN,Cash
8,60 months,13.58%,C,C2,NaN,NaN,MORTGAGE,debt_consolidation,Debt consolidation,894xx,NV,Sep-1990,44.3%,w,Individual,NaN,Cash
9,36 months,6.72%,A,A3,E-7,10+ years,MORTGAGE,debt_consolidation,Debt consolidation,275xx,NC,Oct-2003,27.2%,f,Individual,NaN,Cash


In [16]:
def cleanup1(df1):
    # Start with a fresh copy of the df
    df2 = df1.copy()
    
    # Remove any totally empty columns (there's 3 of them)
    df2 = df2.dropna(axis='columns', how='all')
      
    # Several columns contain percentages mis-parsed as strings. 
    # I'll cast them as floats.
    def parse_percentage_column(col):
        '''
        Parse an iterable and return a list. Turn all the floats into NANs
        and all the strings into floats, after removing the last character.
        This makes sense because these datasets contain nans of type float,
        and the strings have a useless percent sign at the end.
        '''
        new_col = []
        for x in col:
            if type(x) == float:
                new_col.append(np.nan)
            else:
                new_col.append(float(x[:-1]))

        return new_col
    
    df2['int_rate'] = parse_percentage_column(df2['int_rate'])
    df2['revol_util'] = parse_percentage_column(df2['revol_util'])
    
    # There's several columns that contain months since the last [bad thing].
    # I can't think of a good way to impute them, but it looks like a good thing
    # to have a NAN in this column.  I'll create new columns to note people
    # who don't seem to have that bad thing on their record.
    
    
    df2['no_deliq'] = df2.mths_since_last_delinq.isnull().astype(float)
    df2['no_major_derogs'] = df2.mths_since_last_major_derog.isnull().astype(float)
    df2['no_recent_bc_dlq'] = df2.mths_since_recent_bc_dlq.isnull().astype(float)
    df2['no_recent_revol_delinq'] = df2.mths_since_recent_revol_delinq.isnull().astype(float)
    
    # Since I can't figure out how to impute those columns properly, I'll drop them
    df2.drop(columns=['mths_since_last_delinq',
                      'mths_since_last_major_derog',
                      'mths_since_recent_bc_dlq',
                      'mths_since_recent_revol_delinq'])
    
    # Drop all the columns that are mostly NANs, as determined in the training dataset
    mostly_null = [
         'mths_since_last_record',
         'annual_inc_joint',
         'dti_joint',
         'revol_bal_joint',
         'sec_app_earliest_cr_line',
         'sec_app_inq_last_6mths',
         'sec_app_mort_acc',
         'sec_app_open_acc',
         'sec_app_revol_util',
         'sec_app_open_act_il',
         'sec_app_num_rev_accts',
         'sec_app_chargeoff_within_12_mths',
         'sec_app_collections_12_mths_ex_med',
         'sec_app_mths_since_last_major_derog']
    
    df2 = df2.drop(columns=mostly_null)
    
    return df2

In [17]:
train_features.head()

,id,member_id,loan_amnt,funded_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,application_type,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,disbursement_method
0,43373,NaN,12000,12000,36 months,16.02%,422.01,C,C5,Driver,4 years,MORTGAGE,81000.0,NaN,NaN,debt_consolidation,Debt consolidation,280xx,NC,12.76,0,Dec-2005,1,48.0,NaN,7,0,4912,23.5%,15,w,0,48.0,Individual,NaN,NaN,0,0,174783,1,2,0,4,13.0,33668,82.0,2,5,2962,63.0,20900,1,1,2,9,24969.0,588.0,89.3,0,0,145.0,23,1,1,1,8.0,NaN,1.0,NaN,2,2,2,2,2,9,4,5,2,7,0.0,0,0,2,92.9,100.0,0,0,206618,38580,5500,40863,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
1,24414,NaN,6300,6300,36 months,14.07%,215.54,C,C3,GMP Lead,4 years,RENT,39000.0,NaN,NaN,debt_consolidation,Debt consolidation,920xx,CA,21.42,0,Mar-2012,1,NaN,NaN,10,0,11876,59.4%,12,w,0,NaN,Individual,NaN,NaN,0,0,31046,0,1,0,2,14.0,19170,83.0,3,8,5153,72.0,20000,1,0,1,10,3105.0,6877.0,59.3,0,0,17.0,72,8,8,0,8.0,NaN,6.0,NaN,0,6,7,7,8,2,9,10,7,10,0.0,0,0,3,100.0,42.9,0,0,43140,31046,16900,23140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
2,46723,NaN,4500,4500,36 months,7.21%,139.38,A,A3,Accounts,3 years,RENT,78000.0,NaN,NaN,debt_consolidation,Debt consolidation,906xx,CA,2.17,0,May-2000,2,NaN,NaN,13,0,1715,5.2%,19,w,0,NaN,Individual,NaN,NaN,0,0,35329,0,8,1,2,10.0,33614,106.0,1,1,1196,54.0,33300,4,0,3,3,2718.0,21585.0,7.4,0,0,166.0,218,10,10,0,31.0,NaN,1.0,NaN,0,3,3,3,4,13,5,6,3,13,0.0,0,0,2,100.0,0.0,0,0,65092,35329,23300,31792,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
3,24878,NaN,12000,12000,36 months,9.44%,384.06,B,B1,Client Processing Associate,10+ years,MORTGAGE,62000.0,NaN,NaN,debt_consolidation,Debt consolidation,441xx,OH,13.76,0,Feb-2003,0,NaN,NaN,22,0,9404,16.3%,33,w,0,NaN,Individual,NaN,NaN,0,0,75160,4,1,0,2,13.0,17123,67.0,6,9,6898,32.0,57600,3,1,2,12,3579.0,33602.0,17.0,0,0,162.0,179,1,1,1,1.0,NaN,5.0,NaN,0,1,3,9,13,7,20,25,3,22,0.0,0,0,7,100.0,0.0,0,0,133065,26527,40500,25465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
4,39038,NaN,12000,12000,36 months,22.35%,460.47,D,D5,Construction Operator,10+ years,MORTGAGE,70000.0,NaN,NaN,debt_consolidation,Debt consolidation,751xx,TX,14.04,4,Nov-2001,2,13.0,NaN,18,0,17895,24.2%,45,w,0,13.0,Individual,NaN,NaN,0,7218,113702,9,2,2,4,6.0,16711,68.0,9,10,5373,35.0,73900,3,6,3,14,6317.0,32863.0,24.1,0,0,159.0,201,0,0,1,0.0,62.0,3.0,62.0,6,9,11,11,14,22,15,20,11,18,0.0,0,2,11,82.2,9.1,0,0,192842,34606,43300,24499,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash


In [19]:
df3 = cleanup1(train_features)
df4 = cleanup1(test_features)

In [20]:
print(df3.shape)
print(df4.shape)
print(train_features.shape)

(37745, 90)
(9437, 90)
(37745, 103)


## Fit the model

In [41]:
# First, create an array of features X1 that contains only numerical columns with no Nulls.
X2 = cleanup1(train_features).select_dtypes(include='number')
X2.shape

(37745, 76)

In [25]:
# Define the vector y_train of labels for the training data
y_train = train_labels['charged_off']

In [77]:
# Create a pipeline, such that this workflow can be easily expanded in later sections
pipe = make_pipeline(
    SimpleImputer(),
    DecisionTreeClassifier()
)
parameters = {'decisiontreeclassifier__max_depth':[1,3,5,10]}
gs = GridSearchCV(estimator=pipe, 
                  param_grid=parameters, 
                  cv=5, 
                  n_jobs=-1,
                  return_train_score=False)

In [78]:
%%time
gs.fit(X2,y_train)

CPU times: user 580 ms, sys: 99.9 ms, total: 680 ms
Wall time: 6.05 s


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('simpleimputer', SimpleImputer(copy=True, fill_value=None, missing_values=nan, strategy='mean',
       verbose=0)), ('decisiontreeclassifier', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impur...      min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'decisiontreeclassifier__max_depth': [1, 3, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
       scoring=None, verbose=0)

In [79]:
print(f'Best parameters: {gs.best_params_}')
print(f'Best score: {gs.best_score_:0.4f}')

Best parameters: {'decisiontreeclassifier__max_depth': 1}
Best score: 0.8529


## Make and export predictions

In [80]:
# Cleanup test data
x_test = cleanup1(test_features).select_dtypes(include='number')

# Make a prediction, extract results
probs = [x[1] for x in gs.predict_proba(x_test)]

# Export to CSV
actual_submission = sample_submission.copy()
actual_submission.charged_off = probs
actual_submission.to_csv('DMA2.csv', index=False)

In [81]:
actual_submission.head()

,id,charged_off
0,57146,0.240766
1,20988,0.098295
2,10031,0.240766
3,33892,0.098295
4,35374,0.098295
